In [1]:
import mpp_classifiers as mpp
import load_ship_data as lsd
import numpy as np
import time
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold

Get data using our own data loader

In [2]:

data_path="data/shipsnet.json"
train,test,valid=lsd.load_data_train_test_split(data_path)


Training Set Data Length:  2800   Label Length:  2800
TestingSet Set Data Length:  600  Label Length:  600
Validation Set Data Length:  600  Label Length:  600


Setup data splits and priors

In [3]:
ship_prob=np.sum(train[1])*1.0/train[1].shape[0]
p=np.array([1-ship_prob,ship_prob])
print(p)
Xtrain=train[0]
ytrain=train[1]
Xtest=test[0]
ytest=test[1]
Xvalid=valid[0]
yvalid=valid[1]
Xtrain=Xtrain.reshape(Xtrain.shape[0],Xtrain.shape[1]*Xtrain.shape[2]*Xtrain.shape[3])
Xtest=Xtest.reshape(Xtest.shape[0],Xtest.shape[1]*Xtest.shape[2]*Xtest.shape[3])
Xvalid=Xvalid.reshape(Xvalid.shape[0],Xvalid.shape[1]*Xvalid.shape[2]*Xvalid.shape[3])
print(Xtrain.shape)
print(Xtest.shape)
print(Xvalid.shape)

[0.75321429 0.24678571]
(2800, 19200)
(600, 19200)
(600, 19200)


Case 1. No PCA, even priors

In [4]:

p55=np.array([.5,.5])
t0 = time.time()           # start time
y_model = mpp.mpp(Xtrain, ytrain, Xtest, 1, p55)
t1 = time.time()           # ending time
acc_classwise, acc_overall = mpp.accuracy_score(ytest, y_model)
print(f'Overall accuracy = {acc_overall};')
print(f'Classwise accuracy = {acc_classwise};')
print(f'The learning process takes {t1 - t0} seconds.')


Overall accuracy = 0.6566666666666666;
Classwise accuracy = [0.67727273 0.6       ];
The learning process takes 12.190431118011475 seconds.


Case 2. No PCA, prior assumes ships are rare

In [5]:
p91=np.array([.9,.1])
t0 = time.time()           # start time
y_model = mpp.mpp(Xtrain, ytrain, Xtest, 1, p91)
t1 = time.time()           # ending time
acc_classwise, acc_overall = mpp.accuracy_score(ytest, y_model)
print(f'Overall accuracy = {acc_overall};')
print(f'Classwise accuracy = {acc_classwise};')
print(f'The learning process takes {t1 - t0} seconds.')


Overall accuracy = 0.7333333333333333;
Classwise accuracy = [1. 0.];
The learning process takes 11.635922908782959 seconds.


Case 1. No PCA, priors from training data

In [6]:

t0 = time.time()           # start time
y_model = mpp.mpp(Xtrain, ytrain, Xtest, 1, p)
t1 = time.time()           # ending time
acc_classwise, acc_overall = mpp.accuracy_score(ytest, y_model)
print(f'Overall accuracy = {acc_overall};')
print(f'Classwise accuracy = {acc_classwise};')
print(f'The learning process takes {t1 - t0} seconds.')


Overall accuracy = 0.7333333333333333;
Classwise accuracy = [1. 0.];
The learning process takes 12.024845838546753 seconds.


PCA = 800 dimensions kept

In [22]:
pca = PCA(n_components=800)
pca.fit(Xtrain)
pca_Xtrain=pca.transform(Xtrain)
pca_Xtest=pca.transform(Xtest)
explained=np.sum(pca.explained_variance_ratio_)
print(explained)

0.9905807748716495


Case 1. PCA=800, training priors used from now on

In [23]:
p=np.array([1-ship_prob,ship_prob])
t0 = time.time()           # start time
y_model = mpp.mpp(pca_Xtrain, ytrain, pca_Xtest, 1, p)
t1 = time.time()           # ending time
acc_classwise, acc_overall = mpp.accuracy_score(ytest, y_model)
print(f'Overall accuracy = {acc_overall};')
print(f'Classwise accuracy = {acc_classwise};')
print(f'The learning process takes {t1 - t0} seconds.')

Overall accuracy = 0.7333333333333333;
Classwise accuracy = [1. 0.];
The learning process takes 0.05781149864196777 seconds.


Case 2. PCA=800

In [24]:
p=np.array([1-ship_prob,ship_prob])
t0 = time.time()           # start time
y_model = mpp.mpp(pca_Xtrain, ytrain, pca_Xtest, 2, p)
t1 = time.time()           # ending time
acc_classwise, acc_overall = mpp.accuracy_score(ytest, y_model)
print(f'Overall accuracy = {acc_overall};')
print(f'Classwise accuracy = {acc_classwise};')
print(f'The learning process takes {t1 - t0} seconds.')

Overall accuracy = 0.915;
Classwise accuracy = [0.93181818 0.86875   ];
The learning process takes 20.488181114196777 seconds.


Case 3. PCA=800

In [25]:
p=np.array([1-ship_prob,ship_prob])
t0 = time.time()           # start time
y_model = mpp.mpp(pca2_Xtrain, ytrain, pca2_Xtest, 3, p)
t1 = time.time()           # ending time
acc_classwise, acc_overall = mpp.accuracy_score(ytest, y_model)
print(f'Overall accuracy = {acc_overall};')
print(f'Classwise accuracy = {acc_classwise};')
print(f'The learning process takes {t1 - t0} seconds.')

Overall accuracy = 0.8283333333333334;
Classwise accuracy = [0.81818182 0.85625   ];
The learning process takes 0.8118290901184082 seconds.


PCA =200 dimensions kept

In [26]:
pca2 = PCA(n_components=200)
pca2.fit(Xtrain)
pca2_Xtrain=pca2.transform(Xtrain)
pca2_Xtest=pca2.transform(Xtest)
explained=np.sum(pca2.explained_variance_ratio_)
print(explained)

0.9421827530207556


Case 1. PCA=200

In [27]:
p=np.array([1-ship_prob,ship_prob])
t0 = time.time()           # start time
y_model = mpp.mpp(pca2_Xtrain, ytrain, pca2_Xtest, 1, p)
t1 = time.time()           # ending time
acc_classwise, acc_overall = mpp.accuracy_score(ytest, y_model)
print(f'Overall accuracy = {acc_overall};')
print(f'Classwise accuracy = {acc_classwise};')
print(f'The learning process takes {t1 - t0} seconds.')

Overall accuracy = 0.7366666666666667;
Classwise accuracy = [1.     0.0125];
The learning process takes 0.01992011070251465 seconds.


Case 2. PCA=200

In [28]:
p=np.array([1-ship_prob,ship_prob])
t0 = time.time()           # start time
y_model = mpp.mpp(pca2_Xtrain, ytrain, pca2_Xtest, 2, p)
t1 = time.time()           # ending time
acc_classwise, acc_overall = mpp.accuracy_score(ytest, y_model)
print(f'Overall accuracy = {acc_overall};')
print(f'Classwise accuracy = {acc_classwise};')
print(f'The learning process takes {t1 - t0} seconds.')

Overall accuracy = 0.91;
Classwise accuracy = [0.95681818 0.78125   ];
The learning process takes 0.41385936737060547 seconds.


Case 3. PCA=200

In [29]:
p=np.array([1-ship_prob,ship_prob])
t0 = time.time()           # start time
y_model = mpp.mpp(pca2_Xtrain, ytrain, pca2_Xtest, 3, p)
t1 = time.time()           # ending time
acc_classwise, acc_overall = mpp.accuracy_score(ytest, y_model)
print(f'Overall accuracy = {acc_overall};')
print(f'Classwise accuracy = {acc_classwise};')
print(f'The learning process takes {t1 - t0} seconds.')

Overall accuracy = 0.8283333333333334;
Classwise accuracy = [0.81818182 0.85625   ];
The learning process takes 0.8586685657501221 seconds.


Join training and validation since upcoming cross-validation will do this split for us

In [31]:
pca2 = PCA(n_components=200)
Xtrain=np.concatenate((Xtrain,Xvalid))
ytrain=np.concatenate((ytrain,yvalid))
pca2.fit(Xtrain)
pca2_Xtrain=pca2.transform(Xtrain)
pca2_Xtest=pca2.transform(Xtest)
explained=np.sum(pca2.explained_variance_ratio_)
print(explained)

0.9402989447912148


Setup for Kfold cross-validation

In [32]:
p=np.array([1-ship_prob,ship_prob])
X=pca2_Xtrain
y=ytrain
k=10

Case 1

In [33]:

kf=KFold(n_splits=k,random_state=None)
acc_scores=[]
for train_index,test_index in kf.split(X):
    X_train,X_test=X[train_index,:],X[test_index,:]
    y_train,y_test=y[train_index],y[test_index]
    y_model = mpp.mpp(X_train, y_train, X_test, 1, p)
    acc_classwise, acc_overall = mpp.accuracy_score(y_test, y_model)
    acc_scores.append(acc_overall)
avg_acc=sum(acc_scores)*1.0/k
print('Accuracy from each fold =  {}'.format(acc_scores))
print('Average Accuracy = {}'.format(avg_acc))
y_model = mpp.mpp(X_train, y_train, X_test, 1, p)
acc_classwise, acc_overall = mpp.accuracy_score(y_test, y_model)
print(acc_overall)

Accuracy from each fold =  [0.7375, 0.725, 0.785, 0.76, 0.765, 0.785, 0.73, 0.75, 0.77, 0.745]
Average Accuracy = 0.75525
0.745


Case 2

In [34]:
kf=KFold(n_splits=k,random_state=None)
acc_scores=[]
for train_index,test_index in kf.split(X):
    X_train,X_test=X[train_index,:],X[test_index,:]
    y_train,y_test=y[train_index],y[test_index]
    y_model = mpp.mpp(X_train, y_train, X_test, 2, p)
    acc_classwise, acc_overall = mpp.accuracy_score(y_test, y_model)
    acc_scores.append(acc_overall)
avg_acc=sum(acc_scores)*1.0/k
print('Accuracy from each fold =  {}'.format(acc_scores))
print('Average Accuracy = {}'.format(avg_acc))
y_model = mpp.mpp(X_train, y_train, X_test, 2, p)
acc_classwise, acc_overall = mpp.accuracy_score(y_test, y_model)
print(acc_overall)



Accuracy from each fold =  [0.895, 0.9275, 0.915, 0.94, 0.9125, 0.9625, 0.935, 0.945, 0.93, 0.935]
Average Accuracy = 0.9297500000000001
0.935


Case 3

In [35]:
kf=KFold(n_splits=k,random_state=None)
acc_scores=[]
for train_index,test_index in kf.split(X):
    X_train,X_test=X[train_index,:],X[test_index,:]
    y_train,y_test=y[train_index],y[test_index]
    y_model = mpp.mpp(X_train, y_train, X_test, 3, p)
    acc_classwise, acc_overall = mpp.accuracy_score(y_test, y_model)
    acc_scores.append(acc_overall)
avg_acc=sum(acc_scores)*1.0/k
print('Accuracy from each fold =  {}'.format(acc_scores))
print('Average Accuracy = {}'.format(avg_acc))
y_model = mpp.mpp(X_train, y_train, X_test, 3, p)
acc_classwise, acc_overall = mpp.accuracy_score(y_test, y_model)
print(acc_overall)



Accuracy from each fold =  [0.8225, 0.8, 0.79, 0.8225, 0.7925, 0.7425, 0.7925, 0.84, 0.81, 0.795]
Average Accuracy = 0.8007500000000001
0.795
